<a href="https://colab.research.google.com/github/Jesimsim/ds-section2-project/blob/main/section2_project_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **아파트 실거래가 예측**

In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import math

# from sklearn. import
# from sklearn.metric import f1_score

#### **필요코드 설치**

In [3]:
# Google Colab을 사용하는 경우 해당 셀을 실행하세요
import sys

if "google.colab" in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pandas-profiling==2.*
    !pip install pdpbox
    !pip install --upgrade xgboost


import xgboost
import warnings

warnings.filterwarnings("ignore")
# xgboost.config.set_config(verbosity=0)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.2 MB 5.2 MB/s 
     |████████████████████████████████| 965 kB 67.1 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.1
    Uninstalling matplotlib-3.1.1:
      Successfully uninstalled matplotlib-3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pdpbox 0.2.1 requires matplotlib==3.1.1, but you have matplotlib 3.5.3 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.1.1-cp37-cp37m-manylinux1_x86_64.whl (13.1 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.3
    Uninstalling matplotlib-3.5.3:
      Successfully uninstalled matplotlib-3.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 2.13.0 requires matplotlib>=3.2.0, but you have matplotlib 3.1.1 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### **데이터 불러오기 / 병합하기 - 아파트데이터**

In [ ]:
# 데이터 불러오기 및 병합

# 데이터 확인
import os
os.listdir('/content/AptALL')

FileNotFoundError: ignored

In [ ]:
forders= os.listdir('/content/AptALL')
%cd /content/AptALL

/content/AptALL


In [ ]:
# 데이터 병합 - 월별
df = pd.DataFrame()

for files in forders:
    df_1 = pd.read_csv(files, encoding='cp949', encoding_errors='ignore',  skiprows = [i for i in range(0, 15) ])
    df = pd.concat([df, df_1], axis=0)

df.to_csv("Apt_2022_all.csv", encoding='utf-8')

In [ ]:
# 병합오류 확인
pd.read_csv('Apt_2017_all.csv', encoding='utf-8', encoding_errors='ignore') #2022(다시병합), 2017(언어 오류, 병합오류). 2013(병합오류)

In [ ]:
#오류데이터프레임 수정 > 업로드 제대로 마칠때까지 기다리기
df = pd.DataFrame()

for files in forders:
    df_1 = pd.read_csv(files, encoding='cp949',  skiprows = [i for i in range(0, 15) ])
    df = pd.concat([df, df_1], axis=0)

df = df.drop(['Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18','Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22'], axis=1)
df.columns
df.to_csv("Apt_2013_all.csv", encoding='utf-8')

In [ ]:
# 데이터 병합 - 총

import os

forders= os.listdir('/content/AptALL')
%cd /content/AptALL

df = pd.DataFrame()

for files in forders:
    df_1 = pd.read_csv(files, encoding='utf-8', encoding_errors='ignore', index_col=0)
    df = pd.concat([df, df_1], axis=0)


df.to_csv("Apt_total.csv", encoding='utf-8')

/content/AptALL


IsADirectoryError: ignored

In [ ]:
# 에러파일 확인
forders= os.listdir('/content/AptALL')
%cd /content/AptALL
pd.read_csv('Apt_2013_all.csv', encoding='utf-8', encoding_errors='ignore') #2013 unnamed:칼럼 2개


/content/AptALL


,Unnamed: 0,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지
0,0,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.43,202102,9,"145,000",6,1984,개포로 307,20210223.0,-,-
1,1,서울특별시 강남구 개포동,185,185.0,0.0,개포주공6단지,53.06,202102,1,"200,000",12,1983,개포로 516,NaN,-,-
2,2,서울특별시 강남구 개포동,185,185.0,0.0,개포주공6단지,73.02,202102,28,"240,500",7,1983,개포로 516,NaN,-,-
3,3,서울특별시 강남구 개포동,649,649.0,0.0,경남2차,182.20,202102,5,"327,000",11,1984,언주로 110,NaN,-,-
4,4,서울특별시 강남구 개포동,649,649.0,0.0,경남2차,186.16,202102,6,"332,000",14,1984,언주로 110,NaN,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43413,1438,서울특별시 중랑구 신내동,650,650.0,0.0,신내6대주,59.76,202111,30,"75,000",12,1996,신내로19길 42,NaN,중개거래,서울 중랑구
43414,1439,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,59.49,202111,5,"80,000",20,2014,신내역로1길 85,NaN,중개거래,"서울 동대문구, 서울 중랑구"
43415,1440,서울특별시 중랑구 신내동,654,654.0,0.0,진로아파트,101.86,202111,4,"100,000",7,1995,신내로17길 41,NaN,중개거래,서울 중랑구
43416,1441,서울특별시 중랑구 신내동,656,656.0,0.0,화성.두산아파트,84.99,202111,22,"86,000",14,1995,신내로 155,NaN,중개거래,서울 중랑구


In [ ]:
# 에러파일 수정
df = pd.read_csv('Apt_2013_all.csv', encoding='utf-8', encoding_errors='ignore')
df = df.drop('Unnamed: 0.1', axis=1)
df.to_csv("Apt_2013_all(2).csv", encoding='utf-8')

In [ ]:
# 에러파일 확인 2 / 언어오류
df = pd.DataFrame()
forder = [
  'Apt_2019_all.csv',
 'Apt_2013_all.csv',
 'Apt_2017_all.csv',
 'Apt_2010_all.csv',
 'Apt_2011_all.csv',
 'Apt_2022_all.csv',
 'Apt_2015_all.csv',
 'Apt_2012_all.csv',
 'Apt_2020_all.csv',
 'Apt_2018_all.csv',
 'Apt_2021_all.csv',
 'Apt_2014_all.csv',
 'Apt_2016_all.csv']


for files in forders:
    df_1 = pd.read_csv(files, encoding='utf-8', encoding_errors='ignore')
    df = pd.concat([df, df_1], axis=0)

df.head()
# df.to_csv("Apt_check1.csv", encoding='utf-8')

,Unnamed: 0,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지,Unnamed: 0.1
0,0,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),168.42,201912,11,"202,000",5.0,2004.0,개포로109길 69,NaN,-,-,NaN
1,1,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),170.32,201912,11,"218,000",9.0,2004.0,개포로109길 69,NaN,-,-,NaN
2,2,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,41.98,201912,3,"204,000",5.0,1982.0,개포로 310,NaN,-,-,NaN
3,3,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,50.38,201912,4,"220,000",2.0,1982.0,개포로 310,NaN,-,-,NaN
4,4,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,41.98,201912,5,"205,000",1.0,1982.0,개포로 310,NaN,-,-,NaN


In [ ]:
# 오류 확인
pd.read_csv('Apt_total.csv', encoding='utf-8', encoding_errors='ignore')

,index,Unnamed: 0,Unnamed: 0.1,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지,Unnamed: 0.1.1
0,0,0,0,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),168.42,201912,11,"202,000",5.0,2004.0,개포로109길 69,NaN,-,-,NaN
1,1,1,1,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),170.32,201912,11,"218,000",9.0,2004.0,개포로109길 69,NaN,-,-,NaN
2,2,2,2,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,41.98,201912,3,"204,000",5.0,1982.0,개포로 310,NaN,-,-,NaN
3,3,3,3,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,50.38,201912,4,"220,000",2.0,1982.0,개포로 310,NaN,-,-,NaN
4,4,4,4,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,41.98,201912,5,"205,000",1.0,1982.0,개포로 310,NaN,-,-,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924455,924455,110317,10477,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),59.76,201604,21,"30,000",7.0,1997.0,동일로 752,NaN,-,-,NaN
924456,924456,110318,10478,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),84.03,201604,21,"39,000",15.0,1997.0,동일로 752,NaN,-,-,NaN
924457,924457,110319,10479,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),59.76,201604,25,"32,000",21.0,1997.0,동일로 752,NaN,-,-,NaN
924458,924458,110320,10480,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),50.37,201604,26,"27,200",6.0,1997.0,동일로 752,NaN,-,-,NaN


- 파일 가져올때 https://hyunsitstory.tistory.com/entry/python-Jupyter-폴더-내-여러-개의-csv파일을-하나의-데이터-프레임으로-한-번에-합치기
- https://boleumdal.tistory.com/entry/python-폴더-안에-있는-파일-한번에-불러오기 
- 액셀 열때, 텍스트가져오기 마법사 https://support.microsoft.com/ko-kr/office/%ED%85%8D%EC%8A%A4%ED%8A%B8-%EA%B0%80%EC%A0%B8%EC%98%A4%EA%B8%B0-%EB%A7%88%EB%B2%95%EC%82%AC-c5b02af6-fda1-4440-899f-f78bafe41857


#### **데이터불러오기**

In [9]:
df = pd.read_csv('Apt_total.csv', encoding='utf-8', encoding_errors='ignore', engine='python')
df = pd.DataFrame(df, columns=['시군구','번지','본번','부번','단지명','전용면적(㎡)','계약년월'
    ,'계약일','거래금액(만원)','층','건축년도','도로명','해제사유발생일','거래유형','중개사소재지'])
df['거래금액(만원)'] = df['거래금액(만원)'].str.replace(',','')

In [4]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)


In [5]:
df.head()

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지
0,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),168.42,201912,11,"202,000",5.0,2004.0,개포로109길 69,NaN,-,-
1,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),170.32,201912,11,"218,000",9.0,2004.0,개포로109길 69,NaN,-,-
2,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,41.98,201912,3,"204,000",5.0,1982.0,개포로 310,NaN,-,-
3,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,50.38,201912,4,"220,000",2.0,1982.0,개포로 310,NaN,-,-
4,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,41.98,201912,5,"205,000",1.0,1982.0,개포로 310,NaN,-,-


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355657 entries, 0 to 355656
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   시군구       355657 non-null  object 
 1   번지        355535 non-null  object 
 2   본번        355629 non-null  float64
 3   부번        355629 non-null  float64
 4   단지명       355657 non-null  object 
 5   전용면적(㎡)   355656 non-null  float64
 6   계약년월      355656 non-null  float64
 7   계약일       355656 non-null  float64
 8   거래금액(만원)  355656 non-null  object 
 9   층         355655 non-null  float64
 10  건축년도      355654 non-null  float64
 11  도로명       355656 non-null  object 
 12  해제사유발생일   382 non-null     float64
 13  거래유형      355654 non-null  object 
 14  중개사소재지    355654 non-null  object 
dtypes: float64(8), object(7)
memory usage: 40.7+ MB


In [10]:
df.columns

Index(['시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '계약년월', '계약일', '거래금액(만원)',
       '층', '건축년도', '도로명', '해제사유발생일', '거래유형', '중개사소재지'],
      dtype='object')

In [ ]:
df['거래금액(만원)'] = df['거래금액(만원)'].str.replace(',','')

In [13]:
train = df[df['계약년월'] <= 201912]
test = df[df['계약년월'] > 201912]

target = '거래금액(만원)'
feature = '전용면적(㎡)'

X_train = train[feature] 
y_train = train[target]
X_test = test[feature]  
y_test = test[target]

In [14]:
#단순선형회귀모델
from category_encoders import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer


#모델학습
pipe = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    StandardScaler(),
    LinearRegression()
)

pipe.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder', OrdinalEncoder(cols=[], mapping=[])),
                ('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 모델 정확도 계산 / .score()
# R2(결정계수)는 예측값 도출하지 않아도(.predict) 바로 계산 가능
score = pipe.score(X_train, y_train) 
print(f'훈련 에러(R2): {score:.2f}') 
score = pipe.score(X_test, y_test) 
print(f'테스트 에러(R2): {score:.2f}')


# 지표계산 및 출력 / 인자
y_pred_train = pipe.predict(X_train) 
y_pred_test = pipe.predict(X_test)

mae_train = mean_absolute_error(y_train, y_pred_train)  #훈련셋의 MAE
mae_test = mean_absolute_error(y_test, y_pred_test) #테스트셋의 MAE

print(f'훈련 에러: {mae_train:.2f}') 
print(f'테스트 에러: {mae_test:.2f}')


# 지표계산 및 출력 / 데이터프레임
mse = mean_squared_error(y_test, y_pred_test)
mae = mean_absolute_error(y_test, y_pred_test)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred_test)

evaluation1 = pd.DataFrame([['MSE', round(mse, 2)],['MAE', round(mae,2)],['RMSE', round(rmse,2)],['R2', round(r2,2)]], columns=['Metric', 'Score']) 
evaluation1
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred_test)

evaluation1 = pd.DataFrame([['MSE', round(mse, 2)],['MAE', round(mae,2)],['RMSE', round(rmse,2)],['R2', round(r2,2)]], columns=['Metric', 'Score']) 
evaluation1

# 단순선형회귀모델
# 훈련 에러(R2): 0.39
# 테스트 에러(R2): -0.00
# 훈련 에러: 20695.60
# 테스트 에러: 43263.18

# Metric	Score
# 0	MSE	4.556139e+09
# 1	MAE	4.326318e+04
# 2	RMSE	6.749917e+04
# 3	R2	-0.000000e+00


# 다중선형회귀모델
# 시계열로 나눠서 그런지 훈련에러와 테스트에러 차이가 너무 크게 나오네... 다른 모델로 구현해도 그럴까?
# 훈련 에러(R2): 0.45 
# 테스트 에러(R2): 0.26
# 훈련 에러: 19769.78

# 테스트 에러: 34129.16
# Metric	Score
# 0	MSE	3.355991e+09
# 1	MAE	3.412916e+04
# 2	RMSE	5.793092e+04
# 3	R2	2.600000e-01

훈련 에러(R2): 0.39
테스트 에러(R2): -0.00
훈련 에러: 20695.60
테스트 에러: 43263.18


,Metric,Score
0,MSE,4.556139e+09
1,MAE,4.326318e+04
2,RMSE,6.749917e+04
3,R2,-0.000000e+00
